In [12]:
import multiprocessing
import pandas as pd
import numpy as np
import scipy.stats as sc
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.collections import PatchCollection
import dask.dataframe as dd
import os

multiprocessing.cpu_count()

16

In [23]:
np.version.version

'1.19.2'

## Reading tree data, allometric equations, etc... ##
Read in the data and clean it up

 We will first use allometric equations from :

 McPherson, E. Gregory; van Doorn, Natalie S.; Peper, Paula J. 2016. Urban tree database.
 Fort Collins, CO: Forest Service Research Data Archive. Updated 21 January 2020.
 https://doi.org/10.2737/RDS-2016-0005

 'Apps min' and 'Apps max' give the input range (cm) that the authors feel 
  that the equations are reliable
 'InlEmp' and 'SoCalC' are Climate zones where the eqs are different.
  SoCalC reference city is Santa Monica, InlEmp is Claremont,
  see Table 1, p16 for further Climate zone details.  
  
  After reading the equations and coefficients, we will get rid of trees that only occur a few times, and trees that we o not have equations for.

In [14]:
# The equations
def mcpherson_eqs():
    '''returns dict of equations from table 3 (p24) of McPherson 2020
    functions use np so as to be vectorized'''

    eq_dict = {'lin'        : (lambda a, b, c, d, e, x, mse: a + b * (x)), 
                'quad'      : (lambda a, b, c, d, e, x, mse: a + b * x + c * x**2),
                'cub'      : (lambda a, b, c, d, e, x, mse: a + b * x + c * x**2 + d * x**3),
                'quart'     : (lambda a, b, c, d, e, x, mse:a + b * x + c *x**2 + d * x**3 + e * x**4), 
                'loglogw1' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1) + (mse/2)))),
                'loglogw2' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (np.sqrt(x) + (mse/2)))),
                'loglogw3' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (x) + (mse/2))),
                'loglogw4' : (lambda a, b, c, d, e, x, mse: np.exp(a + b * np.log(np.log(x + 1)) + (x**2) + (mse/2))),
                'expow1'    : (lambda a, b, c, d, e, x, mse: np.exp(a+ b * (x) + (mse/2))),
                'expow2'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + np.sqrt(x) + (mse/2))),
                'expow3'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + (x) + (mse/2))),
                'expow4'    : (lambda a, b, c, d, e, x, mse: np.exp(a + b * (x) + (x**2) + (mse/2)))}

    return(eq_dict)

eq_dict = mcpherson_eqs()

# The cooeficients
coef_df = pd.read_csv('TS6_Growth_coefficients.csvx',
usecols=['Region', 'Scientific Name', 'Independent variable', 'Predicts component ', 'EqName', 'Units of predicted components',
'EqName', 'a', 'b', 'c', 'd', 'e', 'Apps min', 'Apps max'])

# Find all the trees with over 100 occurances in the dataset
trees = la.Name_matched.value_counts()
trees = list(trees.where(trees > 100).dropna().index)

# drop trees we do not have equations for
trees = [s for s in trees if s in coef_df['Scientific Name'].unique()]
la = la.loc[la.Name_matched.isin(trees)]

la.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 664155 entries, 0 to 1089845
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   ID            664155 non-null  int64         
 1   LATITUDE      664155 non-null  float64       
 2   LONGITUDE     664155 non-null  float64       
 3   SOURCE        664155 non-null  object        
 4   Name_matched  664155 non-null  object        
 5   Zone          663384 non-null  float64       
 6   dbh_low       664155 non-null  float64       
 7   dbh_high      664155 non-null  float64       
 8   created       28472 non-null   datetime64[ns]
 9   updated       28472 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(1), object(2)
memory usage: 55.7+ MB


## Accessing Lidar data ## 
We will be using the ```USGS_LPC_CA_LosAngeles_2016_LAS_2018``` dataset.  The USGS lidar data is hosted in an Amazon S3 bucket, so we will need the AWS client to access it.

In [15]:
#!curl "https://s3.amazonaws.com/aws-cli/awscli-bundle.zip" -o "awscli-bundle.zip"
#!unzip awscli-bundle.zip 
#!./awscli-bundle/install -b ~/bin/aws

Let's make a tmp directory too, if we don't already have one.

In [16]:
# make a tmp directory

!mkdir ~/tmp

# make a variable for its path
tmp = f'{home}/tmp'

# make a variable with the path to aws cli
aws = '~/bin/aws'

mkdir: cannot create directory ‘/home/jovyan/tmp’: File exists


### Fetching the ept metadata ###
The top level ept json for the ```USGS_LPC_CA_LosAngeles_2016_LAS_2018``` dataset contains important metadata. We will download it.

In [17]:
import subprocess
cmd = f'{aws} s3 cp s3://usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json {tmp} --no-sign-request'
subprocess.run(cmd, shell=True, capture_output=True)

CompletedProcess(args='~/bin/aws s3 cp s3://usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json /home/jovyan/tmp --no-sign-request', returncode=0, stdout=b'Completed 2.4 KiB/2.4 KiB (7.8 KiB/s) with 1 file(s) remaining\rdownload: s3://usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json to ../tmp/ept.json\n', stderr=b'')

We will load ```ept.json``` and extract usefull information.

In [18]:
import json
with open(f'{tmp}/ept.json') as f:
    meta = json.load(f)
    
bounds = meta['bounds']
bounds_conf = meta['boundsConforming']
srs = meta['srs']
span   = meta['span']
schema  = meta['schema']
dataType = meta['dataType']
hierarchyType = meta['hierarchyType']

def bag_scale_offset(name, schema):
    '''Retruns scale and offset for the spatial dimension given by name'''
    for thing in schema:
        if thing['name'] == name:
            return(thing['scale'], thing['offset'])
        
x_scale, x_offset = bag_scale_offset('X', schema)
y_scale, y_offset = bag_scale_offset('Y', schema)
z_scale, z_offset = bag_scale_offset('Z', schema)

#print some info
print(f'dataType is: {dataType}\nhierarchyType is: {hierarchyType}\nsrs is:\n{srs}')

dataType is: laszip
hierarchyType is: json
srs is:
{'authority': 'EPSG', 'horizontal': '3857', 'wkt': 'PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]'}


The above output tells us the data is in EPSG:3857.  There is only a horizontal code present.  Lets reduce the srs to a more useful form for later

In [19]:
srs = meta['srs']['authority'] + ':' + meta['srs']['horizontal']
srs

'EPSG:3857'

Because the srs is in Pseudo-Mercator (EPSG:3857), and our data is lat, lon (EPSG:4326) We will have to reproject. 

### Querying data ###
Now it should be possible to define a bounding box around a tree to query the ept.

In [20]:
from osgeo import osr

# For now we will ad 0.00007 degrees in each direction, this is jus a guess based on 5th decimal place ~ 1.1m
# also not setting z max and min for the moment
def make_scaled_bbox(lat, lon, bounds=None):
    '''Returns a bbox in ept coords.
    If present bounds is of form [xmin, ymin, zmin, xmax, ymax, zmax]'''
    # make the bbox in EPSG:4326
    [xmin, xmax], [ymin, ymax] = make_bbox(lat, lon)
    
    # define transform from EPSG:4326 to EPSG:3857
    old_crs = osr.SpatialReference() 
    old_crs.ImportFromEPSG(4326) 
    new_crs = osr.SpatialReference() 
    new_crs.ImportFromEPSG(3857)
    transform = osr.CoordinateTransformation(old_crs,new_crs)
    
    # transform bbox points
    xmin, ymin, zmin = transform.TransformPoint(ymin, xmin)
    xmax, ymax, zmax = transform.TransformPoint(ymax, xmax)

    # TODO:make sure no bbox is out of the ept bbox, for edge cases
    # if bounds:
        #blah for blah in blah
    
    return([xmin, xmax], [ymin, ymax])

def make_bbox(lat, lon):
    buf = 0.00007
    xmin = lon - buf
    ymin = lat - buf
    xmax = lon + buf
    ymax = lat + buf
    return([xmin, xmax], [ymin, ymax])

def bbox_geojson(lat, lon, filename):
    '''makes wgs84 bbox as geojson for comparison in gis'''
    [xmin, xmax], [ymin, ymax] = make_bbox(lat, lon)
    gjson = {'coordinates' : [[[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]]],
            'type' : 'Polygon'}
    with open(filename, 'w') as of:
        json.dump(gjson, of)

had to run 
```conda update numpy```
in terminal first

In [21]:
pip install pdal

Note: you may need to restart the kernel to use updated packages.


Now lets try to get the point cloud within the bbox using PDALs ept reader

In [22]:
import pdal
from string import Template

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

To test things out we will select a single tree.  ```id``` is the uniqued ID for the tree in the dataset

In [ ]:
# get the lat, lon of point from dataset
uid = 2181656
lat = la.loc[la.ID==uid]['LATITUDE'].values[0]
lon = la.loc[la.ID==uid]['LONGITUDE'].values[0]
species = la.loc[la.ID==uid]['Name_matched'].values[0]

# get bbox coords in EPSG:4326
[xmin, xmax], [ymin, ymax] = make_bbox(lat, lon)

# make geojson of bbox in EPSG:4326
bbox_geojson(lat, lon, f'bbox_{id}.json')

# make transformed bbox
scaled_bbox = make_scaled_bbox(lat, lon, bounds=bounds)

In [ ]:
species
# UTM11 is EPSG:26911

In [ ]:
# make and validate pipeline
t = Template('''
{
    "pipeline": [
        {
        "bounds": "${scaled_bbox}",
        "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json",
        "type": "readers.ept",
        "tag": "readdata",
        "spatialreference":"${srs}"
        },
        {
        "type":"filters.outlier",
        "method":"radius",
        "radius":1.0,
        "min_k":4
        },
        {
        "type": "filters.reprojection",
        "in_srs":"${srs}",
        "out_srs": "EPSG:26911"
        },
        {
        "filename": "dsm${uid}.tif",
        "gdalopts": "tiled=yes,     compress=deflate",
        "nodata": -9999,
        "output_type": "idw",
        "resolution": 1,
        "type": "writers.gdal",
        "window_size": 6
        },
        {
        "type":"filters.smrf",
        "scalar":1.2,
        "slope":0.2,
        "threshold":0.45,
        "window":16.0
        },
        {
        "type":"filters.hag_delaunay"
        },
        {
        "filename": "chm${uid}.tif",
        "gdalopts": "tiled=yes,     compress=deflate",
        "nodata": -9999,
        "output_type": "idw",
        "resolution": 1,
        "type": "writers.gdal",
        "window_size": 6,
        "dimension": "HeightAboveGround"
        }
    ]
}''')

pipe = t.substitute(scaled_bbox=scaled_bbox, srs=srs, uid=uid)

In [ ]:
pipeline = pdal.Pipeline(pipe)

In [ ]:
pipeline.validate()

In [ ]:
count = pipeline.execute()

In [ ]:
count

In [ ]:
S = pipeline.arrays[0]
metadata = pipeline.metadata
log = pipeline.log
S[:4]

So, the pipline made a DSM for the query box.  We also now have the returns in an np structured array, ```S```.

Look at a histogaram of _hag_

In [ ]:
veg = S[S['Classification']!=2]
h = veg['HeightAboveGround'].flatten()

In [ ]:
import matplotlib.pyplot as plt

plt.hist(h, bins=100);

In the historgram above there are a bunch of < 2m returns.  Here is a (huge) picture of the tree (uid = 2181656) from Google street view.  
<img src="streetView_2181656.png">

It seems like those < 2m returns must be from parked cars, or some kind of noise, or some other not-useful thing. Parked cars a re a likely culprit, and they will be trivky to deal with in a systematic way.  For now I am going to just ditch returns below 2m. On small trees this may end up causing issues, we'll see. 

In [ ]:
veg = S[S['HeightAboveGround']>2]
h = veg['HeightAboveGround'].flatten()
plt.hist(h, bins=100);

Good, that looks like half a tree lying on its side!  Next, I write more functions solidifying the work we have done above to make it easiert o do to many trees.  Lets also add some new dimensions to the points using some filters in the PDAL pipeline.

In [ ]:
scaled_bbox

In [ ]:
def make_pipe(bbox, uid, dsm_path, chm_path, srs='EPSG:3857'):
    '''Creates, validates and then returns the pdal pipeline
    
    Arguments:
    bbox     -- Bounding box in srs coordintes (default srs is EPSG:3857),
                in the form: ([xmin, xmax], [ymin, ymax]).
    uid      -- int - ID of the desired tree in the dataframe.
    dsm_path -- String - Path where the DSM shall be saved. Must include .tif exstension.
    chm_path -- String - Path where the CHM shall be saved. Must include .tif exstension.
    srs      -- String - EPSG identifier for srs  being used. Defaults to EPSG:3857
                because that is what ept files tend to use.
    '''
    
    t = Template('''
    {
        "pipeline": [
            {
            "bounds": "${scaled_bbox}",
            "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_LosAngeles_2016_LAS_2018/ept.json",
            "type": "readers.ept",
            "tag": "readdata",
            "spatialreference":"${srs}"
            },
            {
            "type":"filters.outlier",
            "method":"radius",
            "radius":1.0,
            "min_k":4
            },
            {
            "type": "filters.reprojection",
            "in_srs":"${srs}",
            "out_srs": "EPSG:26911"
            },
            {
            "filename": "${dsm_path}",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution": 1,
            "type": "writers.gdal",
            "window_size": 6
            },
            {
            "type":"filters.smrf",
            "scalar":1.2,
            "slope":0.2,
            "threshold":0.45,
            "window":16.0
            },
            {
            "type":"filters.hag_delaunay"
            },
            {
            "filename": "${chm_path}",
            "gdalopts": "tiled=yes,     compress=deflate",
            "nodata": -9999,
            "output_type": "idw",
            "resolution": 1,
            "type": "writers.gdal",
            "window_size": 6,
            "dimension": "HeightAboveGround"
            },
            {
            "type":"filters.optimalneighborhood",
            "min_k":8,
            "max_k": 50
            },
            {
            "type":"filters.covariancefeatures",
            "knn":10,
            "threads": 2,
            "feature_set": "all"
            }
        ]
    }''')

    pipe = t.substitute(scaled_bbox=bbox, srs=srs, uid=uid, dsm_path=dsm_path, chm_path=chm_path)
    pipeline = pdal.Pipeline(pipe)
    if pipeline.validate():
        return(pipeline)
    else:
        raise Exception('Bad pipeline (sorry to be so ambigous)!')

In [ ]:
dsm_path = f'{tmp}/dsm{uid}.tif'
chm_path = f'{tmp}/chm{uid}.tif'


pipeline = make_pipe(scaled_bbox, uid, dsm_path, chm_path)
count = pipeline.execute()
S = pipeline.arrays[0]
metadata = pipeline.metadata
log = pipeline.log
veg = S[(S['Classification']!=2) & (S['HeightAboveGround']>2)]

veg = veg[['Intensity',
           'NumberOfReturns',
           'ReturnNumber',
           'X',
           'Y',
           'HeightAboveGround',
           'OptimalKNN',
           'OptimalRadius', 
           'Linearity', 
           'Planarity', 
           'Scattering', 
           'Verticality', 
           'Omnivariance', 
           'Anisotropy', 
           'Eigenentropy', 
           'EigenvalueSum', 
           'SurfaceVariation', 
           'DemantkeVerticality',
           'Density' ]]

In [ ]:
veg[:2]

Make it a dataframe for convenience.

In [ ]:
df = pd.DataFrame(veg)
df.head()

Find the centroid of highest n points. Then change x and y to be relative to the centroid. Compute radius from that centroid.

In [ ]:
n = 5
tops = df[['X', 'Y', 'HeightAboveGround']].sort_values(ascending=False, by='HeightAboveGround')
x = tops.head(n).X.mean()
y = tops.head(n).Y.mean()

df['X'] = df.X - x
df['Y'] = df.Y - y
df['r'] = np.sqrt(df.X**2 + df.Y**2)
df.sort_values(ascending=True, by='r', inplace=True)


Lets see if there is an obvious trend between r and height.

In [ ]:
df.plot.scatter(x='r', y='HeightAboveGround');

Not really.  Make a tiny dataset in the region.

In [ ]:
uids = [2181656, 2181657, 2181697, 2181698, 2181699, 2184893]
tiny_data = la.loc[la.ID.isin(uids)]

Make a loop of it and parrallelize it etc... 

In [ ]:
#from tqdm import tqdm
from dask import delayed
from dask import compute
from dask.diagnostics import ProgressBar


@delayed
def row_bounds_ept_query(data, idx):
    # get lat lon of first entry
    row = data.iloc[idx]
    uid = row['ID']
    lat = row['LATITUDE']
    lon = row['LONGITUDE']

    # make bbox in EPSG:3857
    scaled_bbox = make_scaled_bbox(lat, lon, bounds=bounds_conf)
    
    # sanity check will raise erros if scalled bbox is not in the ept bounds
    assert (scaled_bbox[0][0] > bounds[0]) & (scaled_bbox[0][1] < bounds[3])
    assert (scaled_bbox[1][0] > bounds[1]) & (scaled_bbox[1][1] < bounds[4])
    
    # make and validate pipeline
    dsm_path = f'{tmp}/dsm{uid}.tif'
    chm_path = f'{tmp}/chm{uid}.tif'

    # make and validate pipeline
    pipeline = make_pipe(scaled_bbox, uid, dsm_path, chm_path)
    
    # execute the piepline
    count = pipeline.execute()
    S = pipeline.arrays[0]
    metadata = pipeline.metadata
    log = pipeline.log
    
    #Trim the pointcloud to what we want to use
    veg = S[(S['Classification']!=2) & (S['HeightAboveGround']>2)]

    # do stuff
    trim = count - veg.shape[0]
    count = veg.shape[0]
    if count > 0:
        # make it a dataframe
        df = pd.DataFrame(veg)
        # find the centroid of highest n points. 
        n = 5
        tops = df[['X', 'Y', 'HeightAboveGround']].sort_values(ascending=False, by='HeightAboveGround')
        x = tops.head(n).X.mean()
        y = tops.head(n).Y.mean()
        # change x and y to be relative to the centroid. Compute radius from that centroid.
        df['X'] = df.X - x
        df['Y'] = df.Y - y
        df['r'] = np.sqrt(df.X**2 + df.Y**2)
        # sort by r
        df.sort_values(ascending=True, by='r', inplace=True)
        
        return(df)
        
results = []
for i in range(len(tiny_data)):
    results.append(row_bounds_ept_query(tiny_data, i))
    
with ProgressBar():
    S = compute(*results)

In [ ]:
len(S)

In [ ]:
S[1].plot.scatter(x='r', y='HeightAboveGround');

In [ ]:
S[2].plot.scatter(x='r', y='HeightAboveGround');

In [ ]:
S[3].plot.scatter(x='r', y='HeightAboveGround');

In [ ]:
S[4].plot.scatter(x='r', y='HeightAboveGround');

In [24]:
import gdal

In [25]:
?gdal.BuildVRT

Signature: gdal.BuildVRT(destName, srcDSOrSrcDSTab, **kwargs)
Docstring:
Build a VRT from a list of datasets.
Arguments are :
  destName --- Output dataset name
  srcDSOrSrcDSTab --- an array of Dataset objects or filenames, or a Dataset object or a filename
Keyword arguments are :
  options --- return of gdal.BuildVRTOptions(), string or array of strings
  other keywords arguments of gdal.BuildVRTOptions()
If options is provided as a gdal.BuildVRTOptions() object, other keywords are ignored. 
File:      /opt/conda/lib/python3.8/site-packages/osgeo/gdal.py
Type:      function


In [28]:
import os
os.listdir(tmp)

['chm40.tif',
 'chm50.tif',
 'chm79.tif',
 'chm90.tif',
 'chm74.tif',
 'chm84.tif',
 'chm88.tif',
 '.ipynb_checkpoints']

In [29]:
'acbsb.fork'.endswith('.fork')

True

In [30]:
tmp

'/home/jovyan/tmp'

In [32]:
import rasterio

In [35]:
?rasterio.DatasetReader.read

Docstring:
Read a dataset's raw pixels as an N-d array

This data is read from the dataset's band cache, which means
that repeated reads of the same windows may avoid I/O.

Parameters
----------
indexes : list of ints or a single int, optional
    If `indexes` is a list, the result is a 3D array, but is
    a 2D array if it is a band index number.

out : numpy ndarray, optional
    As with Numpy ufuncs, this is an optional reference to an
    output array into which data will be placed. If the height
    and width of `out` differ from that of the specified
    window (see below), the raster image will be decimated or
    replicated using the specified resampling method (also see
    below).

    *Note*: the method's return value may be a view on this
    array. In other words, `out` is likely to be an
    incomplete representation of the method's results.

    This parameter cannot be combined with `out_shape`.

out_dtype : str or numpy dtype
    The desired output data type. For examp